In [5]:
# 1. Ativa o radar de atualização em tempo real (Hot Reload)
%load_ext autoreload
%autoreload 2

# Visor, GPS e Conexão 
import sys, pandas as pd
from pathlib import Path
import numpy as np


# Configura o visor financeiro
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.4f}'.format

# GPS Dinâmico e Blindado: Acha a raiz do projeto automaticamente
raiz = Path.cwd() if Path.cwd().joinpath('src').exists() else Path.cwd().parent
sys.path.append(str(raiz))

# Conecta os motores
from src.data_ingestion import carregar_base_basileia
from src.feature_engineering import calcular_historico_credito, padronizar_tempo_emprego, criar_variaveis_dummy, imputar_dados_nulos, remover_colunas_toxicas, definir_variavel_alvo


print(f"STATUS: Cockpit alinhado. Rota travada em: {raiz}")

STATUS: Cockpit alinhado. Rota travada em: C:\Users\igorc\OneDrive\Área de Trabalho\basel_risk_modeling


In [6]:
# A Execução do ETL
print("STATUS: Iniciando esteira de processamento...")

# 1. Extração
loan_data = carregar_base_basileia()

#2. Descarte de Ativos Tóxicos (Cortamos tudo com mais de 50% de buracos)
loan_data = remover_colunas_toxicas(loan_data, limite_toxicidade=0.50)

# 3. Tratamento Temporal
loan_data['emp_length_int'] = padronizar_tempo_emprego(loan_data)
loan_data['mths_since_earliest_cr_line'] = calcular_historico_credito(loan_data, 'earliest_cr_line')

# 4. Tratamento de Nulos (O nosso novo motor de Imputação)
loan_data = imputar_dados_nulos(loan_data)

# 5. Criação do Gabarito (Target) e Isolamento de Vazamento de Dados
loan_data = definir_variavel_alvo(loan_data)

# 6. Explosão de Categóricas (Dummies)
colunas_para_binario = [
    'grade', 'sub_grade', 'home_ownership', 'verification_status', 
    'purpose', 'addr_state', 'initial_list_status'
]
loan_data = criar_variaveis_dummy(loan_data, colunas_para_binario)

# 7. Limpeza de RAM (Descarte de colunas processadas)
loan_data = loan_data.drop(columns=['earliest_cr_line', 'emp_length', 'loan_status'])

print(f"STATUS: SUCESSO. Base limpa e lacrada. Formato atual: {loan_data.shape}")

STATUS: Iniciando esteira de processamento...
STATUS: Rota confirmada. Iniciando leitura de loan_data_2007_2014.csv...
SUCESSO: 466285 linhas extraídas para a RAM.
STATUS: SUCESSO. Base limpa e lacrada. Formato atual: (466285, 157)


In [7]:
# Auditoria Financeira de Dados Nulos (NaN)

print("STATUS: Ligando o radar de Dívida Técnica (Valores Nulos)...")

# 1. A Matemática do Rombo
total_linhas = len(loan_data)
nulos_abs = loan_data.isnull().sum()
nulos_perc = (nulos_abs / total_linhas) * 100

# 2. A Construção do Balanço Patrimonial (DataFrame limpo)
relatorio_nulos = pd.DataFrame({
    'Total_Vazios': nulos_abs,
    'Percentual (%)': nulos_perc
})

# 3. O Filtro de Toxicidade: Queremos ver apenas quem tem rombo, do pior (maior %) para o melhor
relatorio_nulos = relatorio_nulos[relatorio_nulos['Total_Vazios'] > 0].sort_values(by='Percentual (%)', ascending=False)

print(f"ALERTA: O radar detectou {len(relatorio_nulos)} variáveis com dados faltantes na fuselagem.\n")
display(relatorio_nulos.head(15)) # Imprime as 15 variáveis mais tóxicas

STATUS: Ligando o radar de Dívida Técnica (Valores Nulos)...
ALERTA: O radar detectou 18 variáveis com dados faltantes na fuselagem.



,Total_Vazios,Percentual (%)
next_pymnt_d,227214,48.7286
tot_cur_bal,70276,15.0715
tot_coll_amt,70276,15.0715
emp_title,27588,5.9166
emp_length_int,21008,4.5054
last_pymnt_d,376,0.0806
revol_util,340,0.0729
collections_12_mths_ex_med,145,0.0311
last_credit_pull_d,42,0.0090
pub_rec,29,0.0062


In [8]:
loan_data['good_bad_loan'].value_counts()

good_bad_loan
1    416078
0     50207
Name: count, dtype: int64